In [1]:
"""Trains CNN model with optimal hyper-parameters."""

import numpy as np
import pickle
import pandas as pd

from keras import backend as K
from keras.models import Model
from keras.layers import Input, MaxPooling1D, Dropout, Activation
from keras.layers import Conv1D, Dense, Flatten, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import  LearningRateScheduler
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split 
from pathlib import Path
import click
from rdkit import Chem
from rdkit import RDLogger
from scipy.interpolate import interp1d

import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
functional_groups = {
    'Acid anhydride': Chem.MolFromSmarts('[CX3](=[OX1])[OX2][CX3](=[OX1])'),
    'Acyl halide': Chem.MolFromSmarts('[CX3](=[OX1])[F,Cl,Br,I]'),
    'Alcohol': Chem.MolFromSmarts('[#6][OX2H]'),
    'Aldehyde': Chem.MolFromSmarts('[CX3H1](=O)[#6,H]'),
    'Alkane': Chem.MolFromSmarts('[CX4;H3,H2]'),
    'Alkene': Chem.MolFromSmarts('[CX3]=[CX3]'),
    'Alkyne': Chem.MolFromSmarts('[CX2]#[CX2]'),
    'Amide': Chem.MolFromSmarts('[NX3][CX3](=[OX1])[#6]'),
    'Amine': Chem.MolFromSmarts('[NX3;H2,H1,H0;!$(NC=O)]'),
    'Arene': Chem.MolFromSmarts('[cX3]1[cX3][cX3][cX3][cX3][cX3]1'),
    'Azo compound': Chem.MolFromSmarts('[#6][NX2]=[NX2][#6]'),
    'Carbamate': Chem.MolFromSmarts('[NX3][CX3](=[OX1])[OX2H0]'),
    'Carboxylic acid': Chem.MolFromSmarts('[CX3](=O)[OX2H]'),
    'Enamine': Chem.MolFromSmarts('[NX3][CX3]=[CX3]'),
    'Enol': Chem.MolFromSmarts('[OX2H][#6X3]=[#6]'),
    'Ester': Chem.MolFromSmarts('[#6][CX3](=O)[OX2H0][#6]'),
    'Ether': Chem.MolFromSmarts('[OD2]([#6])[#6]'),
    'Haloalkane': Chem.MolFromSmarts('[#6][F,Cl,Br,I]'),
    'Hydrazine': Chem.MolFromSmarts('[NX3][NX3]'),
    'Hydrazone': Chem.MolFromSmarts('[NX3][NX2]=[#6]'),
    'Imide': Chem.MolFromSmarts('[CX3](=[OX1])[NX3][CX3](=[OX1])'),
    'Imine': Chem.MolFromSmarts('[$([CX3]([#6])[#6]),$([CX3H][#6])]=[$([NX2][#6]),$([NX2H])]'),
    'Isocyanate': Chem.MolFromSmarts('[NX2]=[C]=[O]'),
    'Isothiocyanate': Chem.MolFromSmarts('[NX2]=[C]=[S]'),
    'Ketone': Chem.MolFromSmarts('[#6][CX3](=O)[#6]'),
    'Nitrile': Chem.MolFromSmarts('[NX1]#[CX2]'),
    'Phenol': Chem.MolFromSmarts('[OX2H][cX3]:[c]'),
    'Phosphine': Chem.MolFromSmarts('[PX3]'),
    'Sulfide': Chem.MolFromSmarts('[#16X2H0]'),
    'Sulfonamide': Chem.MolFromSmarts('[#16X4]([NX3])(=[OX1])(=[OX1])[#6]'),
    'Sulfonate': Chem.MolFromSmarts('[#16X4](=[OX1])(=[OX1])([#6])[OX2H0]'),
    'Sulfone': Chem.MolFromSmarts('[#16X4](=[OX1])(=[OX1])([#6])[#6]'),
    'Sulfonic acid': Chem.MolFromSmarts('[#16X4](=[OX1])(=[OX1])([#6])[OX2H]'),
    'Sulfoxide': Chem.MolFromSmarts('[#16X3]=[OX1]'),
    'Thial': Chem.MolFromSmarts('[CX3H1](=S)[#6,H]'),
    'Thioamide': Chem.MolFromSmarts('[NX3][CX3]=[SX1]'),
    'Thiol': Chem.MolFromSmarts('[#16X2H]')
}


def match_group(mol: Chem.Mol, func_group) -> int:
    if type(func_group) == Chem.Mol:
        n = len(mol.GetSubstructMatches(func_group))
    else:
        n = func_group(mol)
    return 0 if n == 0 else 1

def get_functional_groups(smiles: str) -> dict:
    RDLogger.DisableLog('rdApp.*')
    smiles = smiles.strip().replace(' ', '')
    mol = Chem.MolFromSmiles(smiles)
    if mol is None: 
        return None
    func_groups = list()
    for func_group_name, smarts in functional_groups.items():
        func_groups.append(match_group(mol, smarts))

    return func_groups


def train_model(X_train, y_train, X_test, num_fgs, aug, num, weighted):
    """Trains final model with the best hyper-parameters."""
    # Input
    print(X_train.shape)
    X_train = X_train.reshape(X_train.shape[0], 600, 1)
    print(X_train.shape)
    # Shape of input data.
    input_shape = X_train.shape[1:]
    input_tensor = Input(shape=input_shape)

    # 1st CNN layer.
    x = Conv1D(filters=31,
               kernel_size=(11), 
               strides=1,
               padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2)(x)

    # 2nd CNN layer.
    x = Conv1D(filters=62,
       kernel_size=(11),
       strides=1,
       padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(pool_size=2, strides=2)(x)

    # Flatten layer.
    x = Flatten()(x)

    # 1st dense layer.
    x = Dense(4927, activation='relu')(x)
    x = Dropout(0.48599073736368)(x)

    # 2nd dense layer.
    x = Dense(2785, activation='relu')(x)
    x = Dropout(0.48599073736368)(x)

    # 3rd dense layer.
    x = Dense(1574, activation='relu')(x)
    x = Dropout(0.48599073736368)(x)

    output_tensor = Dense(num_fgs, activation='sigmoid')(x)
    print('Model Construction')
    model = Model(inputs=input_tensor, outputs=output_tensor)
    model.summary()
    optimizer = Adam()

    if weighted == 1:

        def calculate_class_weights(y_true):
            number_dim = np.shape(y_true)[1]
            weights = np.zeros((2, number_dim))
            # Calculates weights for each label in a for loop.
            for i in range(number_dim):
                weights_n, weights_p = (y_train.shape[0]/(2 * (y_train[:,i] == 0).sum())), (y_train.shape[0]/(2 * (y_train[:,i] == 1).sum()))
                # Weights could be log-dampened to avoid extreme weights for extremly unbalanced data.
                weights[1, i], weights[0, i] = weights_p, weights_n

            return weights.T

        def get_weighted_loss(weights):
            def weighted_loss(y_true, y_pred):
                return K.mean((weights[:,0]**(1.0-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
            return weighted_loss

        model.compile(optimizer=optimizer, loss=get_weighted_loss(calculate_class_weights(y_train)))

    else:

        model.compile(optimizer=optimizer, loss='binary_crossentropy')

    def custom_learning_rate_schedular(epoch):
        if epoch < 31:
            return 2.5e-4
        elif 31 <= epoch < 37:
            return 2.5000001187436283e-05
        elif 37 <= epoch < 42:
            return 2.5000001187436284e-06

    callback = [LearningRateScheduler(custom_learning_rate_schedular, verbose=1)]
    print('Start training')
    # Start training.
    history = model.fit(x=X_train, y=y_train, epochs=41, batch_size=41, callbacks=callback)

    prediction = model.predict(X_test)
    return (prediction > 0.5).astype(int)
    


    


2024-12-17 15:45:29.032808: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-17 15:45:29.062107: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734421529.081771 1456072 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734421529.087751 1456072 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-17 15:45:29.109808: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
def interpolate_to_600(spec):
    

    old_x = np.arange(len(spec))
    new_x = np.linspace(min(old_x), max(old_x), 600)

    interp = interp1d(old_x, spec)
    new_spec = interp(new_x)
    return new_spec

def make_msms_spectrum(spectrum):
    msms_spectrum = np.zeros(10000)
    for peak in spectrum:
        peak_pos = int(peak[0]*10)
        if peak_pos >= 10000:
            peak_pos = 9999

        msms_spectrum[peak_pos] = peak[1]
    
    return msms_spectrum




analytical_data = Path("/data/zjh2/multimodal-spectroscopic-dataset-main/data/multimodal_spectroscopic_dataset")
out_path = Path("/home/dwj/icml_guangpu/multimodal-spectroscopic-dataset-main/runs/runs_f_groups/h_nmr")
column = "h_nmr_spectra"
seed = 3245
print("Loading Data")
training_data = None

if column == 'pos_msms':
    column = 'msms_positive_40ev'
elif column == 'neg_msms':
    column = 'msms_negative_40ev'

for i, parquet_file in enumerate(analytical_data.glob("*.parquet")):
    data = pd.read_parquet(parquet_file, columns=[column, 'smiles'])

    if 'msms' in column:
        data[column] = data[column].map(make_msms_spectrum)

    data['func_group'] = data.smiles.map(get_functional_groups)
    data[column] = data[column].map(interpolate_to_600)

    if training_data is None:
        training_data = data
    else:
        training_data = pd.concat((training_data, data))
    del data

    print("Loaded Data: ", i)

train, test = train_test_split(training_data, test_size=0.1, random_state=seed) 

X_train = np.stack(train[column].to_list())
y_train = np.stack(train['func_group'].to_list())
X_test = np.stack(test[column].to_list())
y_test = np.stack(test['func_group'].to_list())


Loading Data
Loaded Data:  0
Loaded Data:  1
Loaded Data:  2
Loaded Data:  3
Loaded Data:  4
Loaded Data:  5
Loaded Data:  6
Loaded Data:  7
Loaded Data:  8
Loaded Data:  9
Loaded Data:  10
Loaded Data:  11
Loaded Data:  12
Loaded Data:  13
Loaded Data:  14
Loaded Data:  15
Loaded Data:  16
Loaded Data:  17
Loaded Data:  18
Loaded Data:  19
Loaded Data:  20
Loaded Data:  21
Loaded Data:  22
Loaded Data:  23
Loaded Data:  24
Loaded Data:  25
Loaded Data:  26
Loaded Data:  27
Loaded Data:  28
Loaded Data:  29
Loaded Data:  30
Loaded Data:  31
Loaded Data:  32
Loaded Data:  33
Loaded Data:  34
Loaded Data:  35
Loaded Data:  36
Loaded Data:  37
Loaded Data:  38
Loaded Data:  39
Loaded Data:  40
Loaded Data:  41
Loaded Data:  42
Loaded Data:  43
Loaded Data:  44
Loaded Data:  45
Loaded Data:  46
Loaded Data:  47
Loaded Data:  48
Loaded Data:  49
Loaded Data:  50
Loaded Data:  51
Loaded Data:  52
Loaded Data:  53
Loaded Data:  54
Loaded Data:  55
Loaded Data:  56
Loaded Data:  57
Loaded Data

In [ ]:
# Train extended model.
prediction = train_model(X_train, y_train, X_test, 37, 'e', 0, 0)

print(f1_score(y_test, prediction, average='micro'))

with open(out_path / "results.pickle", "wb") as file:
    pickle.dump({'pred': prediction, 'tgt': y_test}, file)

(714962, 600)
(714962, 600, 1)
Model Construction


W0000 00:00:1734423152.992564 1456072 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 600, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 600, 31)        │           372 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 600, 31)        │           124 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 600, 31)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 300, 31)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 300, 62)        │        21,204 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 300, 62)        │           248 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 300, 62)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 150, 62)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 9300)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4927)           │    45,826,027 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4927)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2785)           │    13,724,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2785)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1574)           │     4,385,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1574)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 37)             │        58,275 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,015,894 (244.20 MB)

 Trainable params: 64,015,708 (244.20 MB)

 Non-trainable params: 186 (744.00 B)

Start training

Epoch 1: LearningRateScheduler setting learning rate to 0.00025.
Epoch 1/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 3105s 178ms/step - loss: 0.1719 - learning_rate: 2.5000e-04

Epoch 2: LearningRateScheduler setting learning rate to 0.00025.
Epoch 2/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 2869s 165ms/step - loss: 0.1372 - learning_rate: 2.5000e-04

Epoch 3: LearningRateScheduler setting learning rate to 0.00025.
Epoch 3/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 2871s 165ms/step - loss: 0.1205 - learning_rate: 2.5000e-04

Epoch 4: LearningRateScheduler setting learning rate to 0.00025.
Epoch 4/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 3307s 190ms/step - loss: 0.1088 - learning_rate: 2.5000e-04

Epoch 5: LearningRateScheduler setting learning rate to 0.00025.
Epoch 5/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 3118s 179ms/step - loss: 0.0991 - learning_rate: 2.5000e-04

Epoch 6: LearningRateScheduler setting learning rate to 0.00025.
Epoch 6/41
17439/17439 ━━━━━━━━━━━━━━━━━━━━ 2979s 171ms/step - loss: 